# 化合物表示学习和性质预测

在这篇教程中，我们将介绍如何运用图神经网络（GNN）模型来预测化合物的性质。具体来说，我们将演示如何对其进行预训练（pretrain），如何针对下游任务进行模型微调（finetune），并利用最终的模型进行推断（inference）。如果你想了解更多细节，请查阅 "[info graph](https://github.com/PaddlePaddle/PaddleHelix/apps/pretrained_compound/info_graph/README_cn.md)" 和 "[pretrained GNN](https://github.com/PaddlePaddle/PaddleHelix/apps/pretrained_compound/pretrain_gnns/README_cn.md)" 的详细解释.

# 第一部分：预训练

在这一部分，我们将展示如何预训练一个化合物 GNN 模型。本文中的预训练技术是在预训练 GNN 的基础上发展起来的，包括属性遮盖、上下文预测和有监督预训练。
更多细节请查看文件：`pretrain_attrmask.py`，`pretrain_contextpred.py` 和 `pretrain_supervised.py`。

In [1]:
import os
import numpy as np
import sys
sys.path.insert(0, os.getcwd() + "/..")
os.chdir("../apps/pretrained_compound/pretrain_gnns")

PaddleHelix 是构建于 PaddlePaddle 之上的生物计算深度学习框架。

In [2]:
import paddle
import paddle.fluid as fluid
from paddle.fluid.incubate.fleet.collective import fleet
from pahelix.datasets import load_zinc_dataset
from pahelix.featurizers import PreGNNAttrMaskFeaturizer
from pahelix.utils.compound_tools import CompoundConstants
from pahelix.model_zoo import PreGNNAttrmaskModel

[INFO] 2020-12-15 17:28:43,039 [mp_reader.py:   23]:	ujson not install, fail back to use json instead


In [3]:
# switch to paddle static graph mode.
paddle.enable_static()

/home/ol/anaconda2/envs/paddle2.0/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 构建静态图

通常情况下，我们用 Paddle 提供的 `Program` 和 `Executor` 来构建静态图。这里，我们使用 `model_config` 保存模型配置。`PreGNNAttrmaskModel` 是一种无监督的预训练模型，它随机地对某个节点的原子类型进行遮盖，然后再尝试去预测这个原子的类型。同时，我们使用 Adam 优化器并将学习率（learning rate）设置为 0.001。

若要使用GPU进行训练，请取消注释行 `exe = fluid.Executor(fluid.CUDAPlace(0))`。当 `fluid.CPUPlace()` 用于CPU训练。

In [4]:
model_config = {
    "dropout_rate": 0.5,# dropout rate
    "gnn_type": "gin",  # other choices like "gat", "gcn".
    "layer_num": 5,     # the number of gnn layers.
}
train_prog = fluid.Program()
startup_prog = fluid.Program()
with fluid.program_guard(train_prog, startup_prog):
    with fluid.unique_name.guard():
        model = PreGNNAttrmaskModel(model_config=model_config)
        model.forward()
        opt = fluid.optimizer.Adam(learning_rate=0.001)
        opt.minimize(model.loss)

exe = fluid.Executor(fluid.CPUPlace())
# exe = fluid.Executor(fluid.CUDAPlace(0))
exe.run(startup_prog)
print(model.loss)

var mean_0.tmp_0 : LOD_TENSOR.shape(1,).dtype(FP32).stop_gradient(False)


## 数据集加载和特征提取

使用 `PreGNNAttrMaskFeaturizer` 来配合模型 `PreGNNAttrmaskModel`。它继承了用于特征提取的超类 `Featurizer`。`Featurizer` 有两个功能：`gen_features` 用于将一条原始 SMILES 转换为图数据，而 `collate_fn` 用于将图数据的子列表聚合为一个 batch。这里我们采用 Zinc 数据集来进行预训练。

In [6]:
featurizer = PreGNNAttrMaskFeaturizer(
        model.graph_wrapper, 
        atom_type_num=len(CompoundConstants.atom_num_list),
        mask_ratio=0.15)
dataset = load_zinc_dataset("../../../data/chem_dataset/zinc_standard_agent/raw", featurizer=featurizer)
print("dataset num: %s" % (len(dataset)))

dataset num: 1000


## 启动训练

现在我们开始训练 Attrmask 模型。我们仅训练两个 epoch 作为演示，数据加载的过程通过4个 `workers` 进行了加速。然后我们将预训练后的模型保存到 "./model/pretrain_attrmask"，作为下游任务的初始模型。

In [7]:
def train(exe, train_prog, model, dataset, featurizer):
    data_gen = dataset.iter_batch(
            batch_size=256, num_workers=4, shuffle=True, collate_fn=featurizer.collate_fn)
    list_loss = []
    for batch_id, feed_dict in enumerate(data_gen):
        train_loss, = exe.run(train_prog, 
                feed=feed_dict, fetch_list=[model.loss], return_numpy=False)
        list_loss.append(np.array(train_loss).mean())
    return np.mean(list_loss)

for epoch_id in range(2):
    train_loss = train(exe, train_prog, model, dataset, featurizer)
    print("epoch:%d train/loss:%s" % (epoch_id, train_loss))
fluid.io.save_params(exe, './model/pretrain_attrmask', train_prog)

epoch:0 train/loss:4.2393446
epoch:1 train/loss:1.477257


模型预训练的内容到此为止，你可以根据自己的需要对上面的参数进行调整。

# 第二部分：下游任务模型微调（fintune）

下面我们将介绍如何对预训练的模型进行微调来适应下游任务。

更多细节参见 `finetune.py` 文件中的内容。

In [12]:
from pahelix.utils.paddle_utils import load_partial_params
from pahelix.utils.splitters import \
    RandomSplitter, IndexSplitter, ScaffoldSplitter, RandomScaffoldSplitter
from pahelix.datasets import *

from model import DownstreamModel
from featurizer import DownstreamFeaturizer
from utils import calc_rocauc_score

下游任务的数据集通常规模很小，并且面向不同的任务。例如，BBBP 数据集用于预测化合物的血脑屏障通透性；Tox21 数据集用于预测化合物的毒性等。这里我们使用 Tox21 数据集进行演示。

In [14]:
task_names = get_default_tox21_task_names()
# task_names = get_default_sider_task_names()
print(task_names)

['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']


## 构建静态图

这里我们采用和之前一样的方式构建一个静态图模型。注意这里的模型结构的设置应该和预训练模型中的设置保持一致，否则模型加载将会失败。`DownstreamModel` 是一个有监督的 GNN 模型，用于上述 `task_names` 中定义的预测任务。

我们使用 `train_prog` 和 `test_prog` 来保存静态图，用于后续的训练和测试。它们具有相同的网络架构，但某些操作符的功能将发生变化，例如 `Dropout` 和 `BatchNorm`。

In [15]:
model_config = {
    "dropout_rate": 0.5,# dropout rate
    "gnn_type": "gin",  # other choices like "gat", "gcn".
    "layer_num": 5,     # the number of gnn layers.
    "num_tasks": len(task_names), # number of targets to predict for the downstream task.
}
train_prog = fluid.Program()
test_prog = fluid.Program()
startup_prog = fluid.Program()
with fluid.program_guard(train_prog, startup_prog):
    with fluid.unique_name.guard():
        model = DownstreamModel(model_config=model_config)
        model.train()
        opt = fluid.optimizer.Adam(learning_rate=0.001)
        opt.minimize(model.loss)
with fluid.program_guard(test_prog, fluid.Program()):
    with fluid.unique_name.guard():
        model = DownstreamModel(model_config=model_config)
        model.train(is_test=True)

exe = fluid.Executor(fluid.CPUPlace())
# exe = fluid.Executor(fluid.CUDAPlace(0))
exe.run(startup_prog)

[]

## 加载预训练模型

加载预训练阶段得到的模型。这里我们加载模型 "pretrain_attrmask" 作为一个例子。

In [16]:
load_partial_params(exe, './model/pretrain_attrmask', train_prog)

Load parameters from ./model/pretrain_attrmask.


## 数据加载和特征提取

将 `DownstreamFeaturizer` 与 `DownstreamModel` 一起使用。它继承自用于特征提取的超类 `featureizer`。`featureizer` 有两个功能：`gen_features` 用于将一条原始 SMILES 转换为单个图数据，而 `collate_fn` 用于将图数据的子列表聚合为一个 batch。

Tox21 数据集用作下游任务数据集，我们使用 `ScaffoldSplitter` 将数据集拆分为训练/验证/测试集。`ScaffoldSplitter` 首先根据 Bemis-Murcko scaffold 对化合物进行排序，然后从前到后，将参数 `frac_train` 定义的比例的数据作为训练集，将 `frac_valid` 定义的比例的数据作为验证集，其余的作为测试集。`ScaffoldSplitter` 能更好地评价模型对非同分布样本的泛化能力。这里也可以使用其他的拆分器，如 `RandomSplitter`、`RandomScaffoldSplitter` 和 `IndexSplitter`。

In [13]:
featurizer = DownstreamFeaturizer(model.graph_wrapper)
dataset = load_tox21_dataset(
        "../../../data/chem_dataset/tox21/raw", task_names, featurizer=featurizer)
# dataset = load_sider_dataset(
#         "../../../data/chem_dataset/sider/raw", task_names, featurizer=featurizer)

# splitter = RandomSplitter()
splitter = ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.split(
        dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1)
print("Train/Valid/Test num: %s/%s/%s" % (
        len(train_dataset), len(valid_dataset), len(test_dataset)))

RDKit WARNING: [14:16:10] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:16:25] WARNING: not removing hydrogen atom without neighbors


Train/Valid/Test num: 6264/783/784


## 启动训练

处于演示的目的，这里我们只将 attrmask 模型训练了4轮。由于每个下游任务都包含了多个子任务，我们分别计算了每个子任务的roc-auc，在求其均值作为最后的评估标准。

In [14]:
def train(exe, train_prog, model, train_dataset, featurizer):
    data_gen = train_dataset.iter_batch(
        batch_size=64, num_workers=4, shuffle=True, collate_fn=featurizer.collate_fn)
    list_loss = []
    for batch_id, feed_dict in enumerate(data_gen):
        train_loss, = exe.run(train_prog, feed=feed_dict, fetch_list=[model.loss], return_numpy=False)
        list_loss.append(np.array(train_loss).mean())
    return np.mean(list_loss)

def evaluate(exe, test_prog, model, test_dataset, featurizer):
    """
    In the dataset, a proportion of labels are blank. So we use a `valid` tensor
    to help eliminate these blank labels in both training and evaluation phase.
    
    Returns:
        the average roc-auc of all sub-tasks.
    """
    data_gen = test_dataset.iter_batch(
    		batch_size=64, num_workers=4, shuffle=False, collate_fn=featurizer.collate_fn)
    total_pred = []
    total_label = []
    total_valid = []
    for batch_id, feed_dict in enumerate(data_gen):
        pred, = exe.run(test_prog, feed=feed_dict, fetch_list=[model.pred], return_numpy=False)
        total_pred.append(np.array(pred))
        total_label.append(feed_dict['finetune_label'])
        total_valid.append(feed_dict['valid'])
    total_pred = np.concatenate(total_pred, 0)
    total_label = np.concatenate(total_label, 0)
    total_valid = np.concatenate(total_valid, 0)
    return calc_rocauc_score(total_label, total_pred, total_valid)

for epoch_id in range(4):
    train_loss = train(exe, train_prog, model, train_dataset, featurizer)
    val_auc = evaluate(exe, test_prog, model, valid_dataset, featurizer)
    test_auc = evaluate(exe, test_prog, model, test_dataset, featurizer)
    print("epoch:%s train/loss:%s" % (epoch_id, train_loss))
    print("epoch:%s val/auc:%s" % (epoch_id, val_auc))
    print("epoch:%s test/auc:%s" % (epoch_id, test_auc))
# fluid.io.save_params(exe, './model/sider', train_prog)
fluid.io.save_params(exe, './model/tox21', train_prog)

Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:0 train/loss:0.50505453
epoch:0 val/auc:0.619446883905476
epoch:0 test/auc:0.5755580865907087
Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:1 train/loss:0.25283575
epoch:1 val/auc:0.6492427350509836
epoch:1 test/auc:0.6505639462892321
Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:2 train/loss:0.22008401
epoch:2 val/auc:0.6877695463554699
epoch:2 test/auc:0.6832456625548606
Valid ratio: 0.7603235
Task evaluated: 12/12
Valid ratio: 0.7513818
Task evaluated: 12/12
epoch:3 train/loss:0.21583365
epoch:3 val/auc:0.7055511601823229
epoch:3 test/auc:0.6873961667704048


# 第三部分：下游任务模型预测
在这部分，我们将简单介绍如何利用训好的下游任务模型来对给定的 SMILES 序列做预测。

## 构建静态图
这部分跟第二部分的建图部分基本相同。

In [17]:
model_config = {
    "dropout_rate": 0.5,# dropout rate
    "gnn_type": "gin",  # other choices like "gat", "gcn".
    "layer_num": 5,     # the number of gnn layers.
    "num_tasks": len(task_names), # number of targets to predict for the downstream task.
}
inference_prog = fluid.Program()
startup_prog = fluid.Program()
with fluid.program_guard(inference_prog, startup_prog):
    with fluid.unique_name.guard():
        model = DownstreamModel(model_config=model_config)
        model.inference()

exe = fluid.Executor(fluid.CPUPlace())
# exe = fluid.Executor(fluid.CUDAPlace(0))
exe.run(startup_prog)

[]

## 加载训练好的下游任务模型
加载在第二部分中训练好的下游任务模型。

In [18]:
load_partial_params(exe, './model/tox21', inference_prog)

Load parameters from ./model/tox21.


## 开始预测
对给定的 SMILES 序列进行预测。我们直接调用 `DownstreamFeaturizer` 的 `gen_features` 和 `collate_fn` 函数将原始的 SMILES 序列转化为模型的输入。

以 Tox21 数据集为例，我们的下游任务模型可以给出 Tox21 里面的12个子任务的预测。

In [19]:
SMILES="O=C1c2ccccc2C(=O)C1c1ccc2cc(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c2n1"
featurizer = DownstreamFeaturizer(model.graph_wrapper, is_inference=True)
feed_dict = featurizer.collate_fn([featurizer.gen_features({'smiles': SMILES})])
pred, = exe.run(inference_prog, feed=feed_dict, fetch_list=[model.pred], return_numpy=False)
probs = np.array(pred)[0]
print('SMILES:%s' % SMILES)
print('Predictions:')
for name, prob in zip(task_names, probs):
    print("  %s:\t%s" % (name, prob))

SMILES:O=C1c2ccccc2C(=O)C1c1ccc2cc(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c2n1
Predictions:
  NR-AR:	0.017969187
  NR-AR-LBD:	0.012354077
  NR-AhR:	0.029024104
  NR-Aromatase:	0.015708463
  NR-ER:	0.08152088
  NR-ER-LBD:	0.019772632
  NR-PPAR-gamma:	0.013134609
  SR-ARE:	0.09602512
  SR-ATAD5:	0.012249073
  SR-HSE:	0.025706206
  SR-MMP:	0.058807086
  SR-p53:	0.01833228
